In [55]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import pyogrio
import numpy as np
import plotly.io as pio

In [56]:
pio.templates.default = "plotly_dark"

### Settings

In [57]:
sampling_factor = 0.01

output_path = "../../output/zones" # ptasc-2"
output_prefix = "mun_1pct_"

data_path = "../../analysis/data"

### Zoning data

In [58]:
zones = [
    { "id": "munich", "label": "München", "path": ["munich_area.gpkg"] },
    { "id": "umland", "label": "Münchner Umland", "path": ["umland_area.gpkg"] },
    { "id": "mvv", "label": "MVV Gebiet", "path": ["mvv_area.gpkg"] },
    { "id": "mvg_influence", "label": "MVG Einfluss", "path": ["mvg_influence_area.gpkg"] },
    { "id": "mvg_planning", "label": "MVG Planung", "path": ["mvg_planning_area.gpkg"] },
    { "id": "munich+umland", "label": "München & Umland", "path": ["munich_area.gpkg", "umland_area.gpkg"] },
]

df_zones = []
for zone in zones:
    df_partial = pd.concat([
        gpd.read_file("{}/{}".format(data_path, path))[["geometry"]]
        for path in zone["path"]]).dissolve()

    df_partial["zone_id"] = zone["id"]
    df_partial["zone_label"] = zone["label"]
    df_zones.append(df_partial)

df_zones = pd.concat(df_zones)

### Data preparation

In [59]:
# Load home locations
df_homes = pyogrio.read_dataframe("{}/{}homes.gpkg".format(output_path, output_prefix))

# Create a data frame to attach zone tags 
df_home_zones = gpd.sjoin(df_homes, df_zones, predicate = "within")[["household_id", "zone_id"]]

def home_zones(df, zones):
    df_selection = df_home_zones[df_home_zones["zone_id"].isin(zones)]
    return pd.merge(df, df_selection, on = "household_id")

In [60]:
# Load person information
df_persons = pd.read_csv("{}/{}persons.csv".format(output_path, output_prefix), sep = ";")

In [61]:
# Merge household information
df_households = pd.read_csv("{}/{}households.csv".format(output_path, output_prefix), sep = ";")
df_persons = pd.merge(df_persons, df_households, on = "household_id")

In [62]:
import shapely.geometry as sgeo
df_trips = pd.read_csv("{}/eqasim_trips.csv".format(output_path), sep = ";")

origin = gpd.points_from_xy(df_trips["origin_x"], df_trips["origin_y"])
destination = gpd.points_from_xy(df_trips["destination_x"], df_trips["destination_y"])

df_trips["geometry"] = [sgeo.LineString(od) for od in zip(origin, destination)]
df_trips = gpd.GeoDataFrame(df_trips, crs = "EPSG:25832")

In [63]:
# Load trip information
#df_trips = pyogrio.read_dataframe("{}/{}trips.gpkg".format(output_path, output_prefix))
df_trips["trip_index"] = np.arange(len(df_trips))

# Merge in household id
df_trips = pd.merge(df_trips, df_persons[["person_id", "household_id"]])

# Create a data frame to attach zone tags 
# TODO: Should this be within or intersects? Another option is to use the home zone
#df_trip_zones = gpd.sjoin(df_trips[["trip_index", "geometry"]], df_zones, predicate = "within")[[
#    "trip_index", "zone_id"
#]]

df_trip_zones  = df_trips[["person_id", "trip_index"]].copy()
df_trip_zones = pd.merge(df_trip_zones, df_persons[["person_id", "household_id"]], on = "person_id")
df_trip_zones = pd.merge(df_trip_zones, df_home_zones, on = "household_id").drop(columns = ["person_id"])

def trip_zones(df, zones):
    df_selection = df_trip_zones[df_trip_zones["zone_id"].isin(zones)]
    return pd.merge(df, df_selection, on = "trip_index")

In [64]:
# Add tag if person is active
df_persons["is_active"] = df_persons["person_id"].isin(df_trips["person_id"])

In [65]:
# Enrich trips
df_trips["trips"] = 1.0
df_trips["distance_km"] = df_trips["routed_distance"] * 1e-3
df_trips["travel_time_min"] = df_trips["travel_time"] / 60
df_trips["euclidean_distance_km"] = df_trips["euclidean_distance"] * 1e-3

# Definition of purpose
df_trips["purpose"] = df_trips["following_purpose"]
f_return = df_trips["following_purpose"] == "home"
df_trips.loc[f_return, "purpose"] = df_trips.loc[f_return, "preceding_purpose"]

# Remove zero distance trips
df_trips = df_trips[df_trips["euclidean_distance"] > 0]

# MiD Comparison

### Total population

Extracted from Chapter 1

In [66]:
df = home_zones(df_persons[["household_id"]], ["mvv", "munich"]).groupby(
    "zone_id").size().reset_index(name = "population").assign(data = "synthetic")
df["population"] /= sampling_factor

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "population": 2.9 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "population": 1.5 * 1e6, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "population", pattern_shape = "data", barmode = "group",
    title = "Population by perimeter"
)

### Active population (Figure 12)

In [67]:
df = home_zones(df_persons[["household_id", "is_active"]], ["mvv", "munich", "umland"]).groupby(
    "zone_id")["is_active"].mean().reset_index(name = "active_share").assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "active_share": 0.89, "data": "MiD" },
    { "zone_id": "munich", "active_share": 0.95, "data": "MiD" },
    { "zone_id": "umland", "active_share": 0.82, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "active_share", pattern_shape = "data", barmode = "group",
    title = "Share of the active population"
)

### Car availability (Figure 8)

In [68]:
df = home_zones(df_persons[["household_id", "car_availability"]], ["mvv", "munich", "umland"])
df["car_availability"] = df["car_availability"] == "all"
df = df.groupby("zone_id")["car_availability"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "car_availability": 0.69, "data": "MiD" },
    { "zone_id": "munich", "car_availability": 0.57, "data": "MiD" },
    { "zone_id": "umland", "car_availability": 0.83, "data": "MiD" },
    { "zone_id": "mr", "car_availability": 0.47, "data": "MiD" },
    { "zone_id": "mrs", "car_availability": 0.62, "data": "MiD" },

])])

px.bar(
    df, x = "zone_id", y = "car_availability", pattern_shape = "data", barmode = "group",
    title = "Car availability"
)

### Bike availability (Table 2)

In [69]:
df = home_zones(df_persons[["household_id", "bike_availability"]], ["mvv", "munich", "umland"])
df["bike_availability"] = df["bike_availability"] == "all"
df = df.groupby("zone_id")["bike_availability"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "bike_availability": 0.84, "data": "MiD" },
    { "zone_id": "munich", "bike_availability": 0.83, "data": "MiD" },
    { "zone_id": "umland", "bike_availability": 0.87, "data": "MiD" },
    { "zone_id": "mr", "bike_availability": 0.84, "data": "MiD" },
    { "zone_id": "mrs", "bike_availability": 0.83, "data": "MiD" },

])])

px.bar(
    df, x = "zone_id", y = "bike_availability", pattern_shape = "data", barmode = "group",
    title = "Bikes availability"
)

In [70]:
df = home_zones(df_persons[["household_id", "sex", "bike_availability"]], ["munich", "umland"])
df["bike_availability"] = df["bike_availability"] == "all"
df = df.groupby(["zone_id", "sex"])["bike_availability"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "sex": "male", "bike_availability": 0.85, "data": "MiD" },
    { "zone_id": "munich", "sex": "female", "bike_availability": 0.82, "data": "MiD" },
    { "zone_id": "umland", "sex": "male", "bike_availability": 0.88, "data": "MiD" },
    { "zone_id": "umland", "sex": "female", "bike_availability": 0.85, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "bike_availability", pattern_shape = "data", barmode = "group",
    title = "Bike availability by sex", color = "sex"
)

In [71]:
df = home_zones(df_persons[["household_id", "age", "bike_availability"]], ["munich", "umland"])
df["bike_availability"] = df["bike_availability"] == "all"

age_classes = [
    { "age_class": "0-17", "range": (0, 17) },
    { "age_class": "18-29", "range": (18, 29) },
    { "age_class": "30-49", "range": (30, 49) },
    { "age_class": "50-64", "range": (50, 64) },
    { "age_class": "65-74", "range": (65, 74) },
    { "age_class": "75+", "range": (75, np.inf) },
]

for age_class in age_classes:
    df.loc[df["age"].between(*age_class["range"]), "age_class"] = age_class["age_class"]

df = df.groupby(["zone_id", "age_class"])["bike_availability"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "age_class": "0-17", "bike_availability": 0.92, "data": "MiD" },
    { "zone_id": "munich", "age_class": "18-29", "bike_availability": 0.85, "data": "MiD" },
    { "zone_id": "munich", "age_class": "30-49", "bike_availability": 0.90, "data": "MiD" },
    { "zone_id": "munich", "age_class": "50-64", "bike_availability": 0.87, "data": "MiD" },
    { "zone_id": "munich", "age_class": "65-74", "bike_availability": 0.76, "data": "MiD" },
    { "zone_id": "munich", "age_class": "75+", "bike_availability": 0.57, "data": "MiD" },
    { "zone_id": "umland", "age_class": "0-17", "bike_availability": 0.96, "data": "MiD" },
    { "zone_id": "umland", "age_class": "18-29", "bike_availability": 0.80, "data": "MiD" },
    { "zone_id": "umland", "age_class": "30-49", "bike_availability": 0.90, "data": "MiD" },
    { "zone_id": "umland", "age_class": "50-64", "bike_availability": 0.90, "data": "MiD" },
    { "zone_id": "umland", "age_class": "65-74", "bike_availability": 0.85, "data": "MiD" },
    { "zone_id": "umland", "age_class": "75+", "bike_availability": 0.72, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "bike_availability", pattern_shape = "data", barmode = "group",
    title = "Bike availability by age", color = "age_class"
)

### Public transport subscription by region (Figure 11)

In [72]:
df = home_zones(df_persons[["household_id", "has_pt_subscription"]], ["mvv", "munich", "umland"])
df = df.groupby("zone_id")["has_pt_subscription"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "has_pt_subscription": 0.35, "data": "MiD" },
    { "zone_id": "munich", "has_pt_subscription": 0.47, "data": "MiD" },
    { "zone_id": "umland", "has_pt_subscription": 0.22, "data": "MiD" },
    { "zone_id": "mr", "has_pt_subscription": 0.51, "data": "MiD" },
    { "zone_id": "mrs", "has_pt_subscription": 0.45, "data": "MiD" },

])])

px.bar(
    df, x = "zone_id", y = "has_pt_subscription", pattern_shape = "data", barmode = "group",
    title = "PT subscription"
)

### Public transport subscription by sociodemographics (Table 3)

In [73]:
df = home_zones(df_persons[["household_id", "sex", "has_pt_subscription"]], ["munich", "umland"])
df = df.groupby(["zone_id", "sex"])["has_pt_subscription"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "sex": "male", "has_pt_subscription": 0.46, "data": "MiD" },
    { "zone_id": "munich", "sex": "female", "has_pt_subscription": 0.50, "data": "MiD" },
    { "zone_id": "umland", "sex": "male", "has_pt_subscription": 0.23, "data": "MiD" },
    { "zone_id": "umland", "sex": "female", "has_pt_subscription": 0.21, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "has_pt_subscription", pattern_shape = "data", barmode = "group",
    title = "PT Subscription by sex", color = "sex"
)

In [74]:
df = home_zones(df_persons[["household_id", "age", "has_pt_subscription"]], ["munich", "umland"])

age_classes = [
    { "age_class": "0-17", "range": (0, 17) },
    { "age_class": "18-29", "range": (18, 29) },
    { "age_class": "30-49", "range": (30, 49) },
    { "age_class": "50-64", "range": (50, 64) },
    { "age_class": "65-74", "range": (65, 74) },
    { "age_class": "75+", "range": (75, np.inf) },
]

for age_class in age_classes:
    df.loc[df["age"].between(*age_class["range"]), "age_class"] = age_class["age_class"]

df = df.groupby(["zone_id", "age_class"])["has_pt_subscription"].mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "age_class": "0-17", "has_pt_subscription": 0.52, "data": "MiD" },
    { "zone_id": "munich", "age_class": "18-29", "has_pt_subscription": 0.65, "data": "MiD" },
    { "zone_id": "munich", "age_class": "30-49", "has_pt_subscription": 0.48, "data": "MiD" },
    { "zone_id": "munich", "age_class": "50-64", "has_pt_subscription": 0.40, "data": "MiD" },
    { "zone_id": "munich", "age_class": "65-74", "has_pt_subscription": 0.37, "data": "MiD" },
    { "zone_id": "munich", "age_class": "75+", "has_pt_subscription": 0.34, "data": "MiD" },
    { "zone_id": "umland", "age_class": "0-17", "has_pt_subscription": 0.41, "data": "MiD" },
    { "zone_id": "umland", "age_class": "18-29", "has_pt_subscription": 0.39, "data": "MiD" },
    { "zone_id": "umland", "age_class": "30-49", "has_pt_subscription": 0.22, "data": "MiD" },
    { "zone_id": "umland", "age_class": "50-64", "has_pt_subscription": 0.20, "data": "MiD" },
    { "zone_id": "umland", "age_class": "65-74", "has_pt_subscription": 0.11, "data": "MiD" },
    { "zone_id": "umland", "age_class": "75+", "has_pt_subscription": 0.11, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "has_pt_subscription", pattern_shape = "data", barmode = "group",
    title = "Pt Subscrption by age", color = "age_class"
)

### Daily trips, distance, travel time (Figure 12)
The text mentions that this is calculated over all (also inactive) persons.

In [75]:
df = home_zones(
    df_trips[["household_id", "trips", "travel_time_min", "distance_km"]], ["mvv", "munich", "umland"]
).groupby(["household_id", "zone_id"]).sum().reset_index().drop(
    columns = ["household_id"]
).groupby("zone_id").mean().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "trips": 3.2, "travel_time_min": 89, "distance_km": 42, "data": "MiD" },
    { "zone_id": "munich", "trips": 3.2, "travel_time_min": 95, "distance_km": 40, "data": "MiD" },
    { "zone_id": "umland", "trips": 3.1, "travel_time_min": 82, "distance_km": 44, "data": "MiD" },
])])

In [76]:
px.bar(
    df, x = "zone_id", y = "trips", pattern_shape = "data", barmode = "group",
    title = "Daily average trips per person"
)

In [77]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Daily average travel time per person"
)

In [78]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Daily average distance per person"
)

### Median distances and travel times by purposes, mode and globally (Figures 15, 16)

In [79]:
df = trip_zones(df_trips[["trip_index", "purpose", "distance_km", "travel_time_min"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby(
    ["zone_id", "purpose"]).median(numeric_only = True).reset_index().assign(data = "synthetic")
df = df[df["purpose"] != "other"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "purpose": "work", "distance_km": 8, "travel_time_min": 30, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "education", "distance_km": 2, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "shop", "distance_km": 2, "travel_time_min": 10, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "leisure", "distance_km": 4, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "purpose": "work", "distance_km": 6, "travel_time_min": 30, "data": "MiD" },
    { "zone_id": "munich", "purpose": "education", "distance_km": 2, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "munich", "purpose": "shop", "distance_km": 1, "travel_time_min": 10, "data": "MiD" },
    { "zone_id": "munich", "purpose": "leisure", "distance_km": 4, "travel_time_min": 25, "data": "MiD" },
    { "zone_id": "umland", "purpose": "work", "distance_km": 14, "travel_time_min": 30, "data": "MiD" },
    { "zone_id": "umland", "purpose": "education", "distance_km":3, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "purpose": "shop", "distance_km": 2, "travel_time_min": 10, "data": "MiD" },
    { "zone_id": "umland", "purpose": "leisure", "distance_km": 5, "travel_time_min": 20, "data": "MiD" },
])])

In [80]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Median distance by purpose", color = "purpose"
)

In [81]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Median travel time by purpose", color = "purpose"
)

In [82]:
df = trip_zones(df_trips[["trip_index", "mode", "distance_km", "travel_time_min"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby(
    ["zone_id", "mode"]).median(numeric_only = True).reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "distance_km": 1, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "distance_km": 2, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "distance_km": 8, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "distance_km": 8, "travel_time_min": 35, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "distance_km": 1, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "distance_km": 2, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "distance_km": 7, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "distance_km": 6, "travel_time_min": 30, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "distance_km": 1, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "distance_km": 2, "travel_time_min": 10, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "distance_km": 6, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "distance_km": 8, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "distance_km": 14, "travel_time_min": 45, "data": "MiD" },
])])

In [83]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Median distance by mode", color = "mode"
)

In [84]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Median travel time by mode", color = "mode"
)

In [85]:
df = trip_zones(df_trips[["trip_index", "distance_km", "travel_time_min"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby(
    ["zone_id"]).median(numeric_only = True).reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "distance_km": 4, "travel_time_min": 17, "data": "MiD" },
    { "zone_id": "munich", "distance_km": 4, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "umland", "distance_km": 5, "travel_time_min": 15, "data": "MiD" },
])])

In [86]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Median distance"
)

In [87]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Median travel time"
)

### Median travel distances and times by sociodemographics (Table 4)

In [88]:
sociodemgraphics = [
    { "name": "male", "selector": lambda df: df["sex"] == "male" },
    { "name": "female", "selector": lambda df: df["sex"] == "female" },
    { "name": "0-17", "selector": lambda df: df["age"].between(0, 17) },
    { "name": "18-29", "selector": lambda df: df["age"].between(18, 29) },
    { "name": "30-49", "selector": lambda df: df["age"].between(30, 49) },
    { "name": "50-64", "selector": lambda df: df["age"].between(50, 64) },
    { "name": "65-74", "selector": lambda df: df["age"].between(65, 74) },
    { "name": "75+", "selector": lambda df: df["age"].between(75, np.inf) },
]

def sociodemgraphic_groups(df):
    df_sociodemographics = []
    
    for row in sociodemgraphics:
        df_partial = df[row["selector"](df)].copy()
        df_partial["group"] = row["name"]
        df_sociodemographics.append(df_partial)
    
    return pd.concat(df_sociodemographics)

In [89]:
df = sociodemgraphic_groups(pd.merge(df_trips, df_persons[["person_id", "age", "sex"]]))
df = trip_zones(df[["trip_index", "group", "travel_time_min", "distance_km"]], ["munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "group", "zone_id"
]).median().reset_index().assign(data = "synthetic")

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "group": "male", "distance_km": 3.8, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "female", "distance_km": 3.0, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "0-17", "distance_km": 2.0, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "munich", "group": "18-29", "distance_km": 4.5, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "30-49", "distance_km": 3.8, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "50-64", "distance_km": 3.9, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "65-74", "distance_km": 2.9, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "munich", "group": "75+", "distance_km": 2.9, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "umland", "group": "male", "distance_km": 5.7, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "female", "distance_km": 3.8, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "0-17", "distance_km": 2.5, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "18-29", "distance_km": 7.6, "travel_time_min": 20, "data": "MiD" },
    { "zone_id": "umland", "group": "30-49", "distance_km": 5.7, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "50-64", "distance_km": 4.9, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "65-74", "distance_km": 3.5, "travel_time_min": 15, "data": "MiD" },
    { "zone_id": "umland", "group": "75+", "distance_km": 2.9, "travel_time_min": 15, "data": "MiD" },
])])

In [90]:
px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Median distance by group", color = "group"
)

In [91]:
px.bar(
    df, x = "zone_id", y = "travel_time_min", pattern_shape = "data", barmode = "group",
    title = "Median travel time by group", color = "group"
)

### Mode share by trips and distance (Figure 17)

In [92]:
df = trip_zones(df_trips[["trip_index", "mode"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "mode", "zone_id"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("zone_id")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total, on = "zone_id")
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "share": 0.21, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "share": 0.15, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "share": 0.12, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "share": 0.34, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "share": 0.18, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "share": 0.24, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "share": 0.18, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "share": 0.10, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "share": 0.24, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "share": 0.24, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "share": 0.18, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "share": 0.13, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "share": 0.14, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "share": 0.44, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "share": 0.11, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "share", pattern_shape = "data", barmode = "group",
    title = "Mode share by trips", color = "mode"
)

In [93]:
df = trip_zones(df_trips[["trip_index", "mode", "distance_km"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "mode", "zone_id"
])["distance_km"].sum().reset_index(name = "distance").assign(data = "synthetic")

df_total = df.groupby("zone_id")["distance"].sum().reset_index(name = "total")
df = pd.merge(df, df_total, on = "zone_id")
df["share"] = df["distance"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "share": 0.03, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "share": 0.04, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "share": 0.19, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "share": 0.46, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "share": 0.28, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "share": 0.03, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "share": 0.05, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "share": 0.20, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "share": 0.36, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "share": 0.36, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "share": 0.01, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "share": 0.04, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "share": 0.19, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "share": 0.55, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "share": 0.20, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "share", pattern_shape = "data", barmode = "group",
    title = "Mode share by distance", color = "mode"
)

### Mode share by sociodemographic groups (Figure 22)

In MVV perimeter.

In [94]:
df = sociodemgraphic_groups(pd.merge(df_trips, df_persons[["person_id", "age", "sex"]]))
df = trip_zones(df[["trip_index", "group", "mode"]], ["mvv"]).drop(columns = ["trip_index"]).groupby([
    "group", "mode"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("group")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total)
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "group": "male", "mode": "pt", "share": 0.17, "data": "MiD" },
    { "group": "male", "mode": "car_passenger", "share": 0.09, "data": "MiD" },
    { "group": "male", "mode": "car", "share": 0.38, "data": "MiD" },
    { "group": "male", "mode": "bike", "share": 0.15, "data": "MiD" },
    { "group": "male", "mode": "walk", "share": 0.20, "data": "MiD" },
    { "group": "female", "mode": "pt", "share": 0.19, "data": "MiD" },
    { "group": "female", "mode": "car_passenger", "share": 0.15, "data": "MiD" },
    { "group": "female", "mode": "car", "share": 0.29, "data": "MiD" },
    { "group": "female", "mode": "bike", "share": 0.16, "data": "MiD" },
    { "group": "female", "mode": "walk", "share": 0.22, "data": "MiD" },
    { "group": "0-17", "mode": "pt", "share": 0.16, "data": "MiD" },
    { "group": "0-17", "mode": "car_passenger", "share": 0.33, "data": "MiD" },
    { "group": "0-17", "mode": "car", "share": 0.03, "data": "MiD" },
    { "group": "0-17", "mode": "bike", "share": 0.20, "data": "MiD" },
    { "group": "0-17", "mode": "walk", "share": 0.29, "data": "MiD" },
    { "group": "18-29", "mode": "pt", "share": 0.27, "data": "MiD" },
    { "group": "18-29", "mode": "car_passenger", "share": 0.11, "data": "MiD" },
    { "group": "18-29", "mode": "car", "share": 0.31, "data": "MiD" },
    { "group": "18-29", "mode": "bike", "share": 0.14, "data": "MiD" },
    { "group": "18-29", "mode": "walk", "share": 0.17, "data": "MiD" },
    { "group": "30-49", "mode": "pt", "share": 0.17, "data": "MiD" },
    { "group": "30-49", "mode": "car_passenger", "share": 0.07, "data": "MiD" },
    { "group": "30-49", "mode": "car", "share": 0.41, "data": "MiD" },
    { "group": "30-49", "mode": "bike", "share": 0.15, "data": "MiD" },
    { "group": "30-49", "mode": "walk", "share": 0.20, "data": "MiD" },
    { "group": "50-64", "mode": "pt", "share": 0.16, "data": "MiD" },
    { "group": "50-64", "mode": "car_passenger", "share": 0.07, "data": "MiD" },
    { "group": "50-64", "mode": "car", "share": 0.44, "data": "MiD" },
    { "group": "50-64", "mode": "bike", "share": 0.14, "data": "MiD" },
    { "group": "50-64", "mode": "walk", "share": 0.18, "data": "MiD" },
    { "group": "65-74", "mode": "pt", "share": 0.14, "data": "MiD" },
    { "group": "65-74", "mode": "car_passenger", "share": 0.09, "data": "MiD" },
    { "group": "65-74", "mode": "car", "share": 0.37, "data": "MiD" },
    { "group": "65-74", "mode": "bike", "share": 0.14, "data": "MiD" },
    { "group": "65-74", "mode": "walk", "share": 0.25, "data": "MiD" },
    { "group": "75+", "mode": "pt", "share": 0.16, "data": "MiD" },
    { "group": "75+", "mode": "car_passenger", "share": 0.11, "data": "MiD" },
    { "group": "75+", "mode": "car", "share": 0.31, "data": "MiD" },
    { "group": "75+", "mode": "bike", "share": 0.15, "data": "MiD" },
    { "group": "75+", "mode": "walk", "share": 0.27, "data": "MiD" },
])])

In [95]:
px.bar(
    df, x = "group", y = "share", facet_col = "data", barmode = "stack",
    title = "Mode share by group", color = "mode"
)

### Share of purposes (Figure 23)

In [96]:
df = trip_zones(df_trips[["trip_index", "purpose"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "purpose", "zone_id"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("zone_id")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total, on = "zone_id")
df["share"] = df["count"] / df["total"]

df = df[df["purpose"] != "other"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "purpose": "work", "share": 0.18 + 0.1, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "education", "share": 0.07, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "shop", "share": 0.16, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "leisure", "share": 0.29, "data": "MiD" },
    { "zone_id": "munich", "purpose": "work", "share": 0.19 + 0.09, "data": "MiD" },
    { "zone_id": "munich", "purpose": "education", "share": 0.06, "data": "MiD" },
    { "zone_id": "munich", "purpose": "shop", "share": 0.16, "data": "MiD" },
    { "zone_id": "munich", "purpose": "leisure", "share": 0.31, "data": "MiD" },
    { "zone_id": "umland", "purpose": "work", "share": 0.16 + 0.1, "data": "MiD" },
    { "zone_id": "umland", "purpose": "education", "share": 0.08, "data": "MiD" },
    { "zone_id": "umland", "purpose": "shop", "share": 0.15, "data": "MiD" },
    { "zone_id": "umland", "purpose": "leisure", "share": 0.28, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "share", pattern_shape = "data", barmode = "group",
    title = "Purpose share by trips", color = "purpose"
)

### Share of purposes by sociodemographic groups (Figure 25)

In MVV perimeter.

In [97]:
df = sociodemgraphic_groups(pd.merge(df_trips, df_persons[["person_id", "age", "sex"]]))
df = trip_zones(df[["trip_index", "group", "purpose"]], ["mvv"]).drop(columns = ["trip_index"]).groupby([
    "group", "purpose"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("group")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total)
df["share"] = df["count"] / df["total"]

df = df[df["purpose"] != "other"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "group": "male", "purpose": "work", "share": 0.19 + 0.14, "data": "MiD" },
    { "group": "male", "purpose": "education", "share": 0.07, "data": "MiD" },
    { "group": "male", "purpose": "shop", "share": 0.14, "data": "MiD" },
    { "group": "male", "purpose": "leisure", "share": 0.28, "data": "MiD" },
    { "group": "female", "purpose": "work", "share": 0.16 + 0.06, "data": "MiD" },
    { "group": "female", "purpose": "education", "share": 0.07, "data": "MiD" },
    { "group": "female", "purpose": "shop", "share": 0.17, "data": "MiD" },
    { "group": "female", "purpose": "leisure", "share": 0.30, "data": "MiD" },
    { "group": "0-17", "purpose": "work", "share": 0.0, "data": "MiD" },
    { "group": "0-17", "purpose": "education", "share": 0.36, "data": "MiD" },
    { "group": "0-17", "purpose": "shop", "share": 0.05, "data": "MiD" },
    { "group": "0-17", "purpose": "leisure", "share": 0.38, "data": "MiD" },
    { "group": "18-29", "purpose": "work", "share": 0.23 + 0.10, "data": "MiD" },
    { "group": "18-29", "purpose": "education", "share": 0.09, "data": "MiD" },
    { "group": "18-29", "purpose": "shop", "share": 0.12, "data": "MiD" },
    { "group": "18-29", "purpose": "leisure", "share": 0.30, "data": "MiD" },
    { "group": "30-49", "purpose": "work", "share": 0.26 + 0.13, "data": "MiD" },
    { "group": "30-49", "purpose": "education", "share": 0.0, "data": "MiD" },
    { "group": "30-49", "purpose": "shop", "share": 0.14, "data": "MiD" },
    { "group": "30-49", "purpose": "leisure", "share": 0.25, "data": "MiD" },
    { "group": "50-64", "purpose": "work", "share": 0.23 + 0.17, "data": "MiD" },
    { "group": "50-64", "purpose": "education", "share": 0.0, "data": "MiD" },
    { "group": "50-64", "purpose": "shop", "share": 0.17, "data": "MiD" },
    { "group": "50-64", "purpose": "leisure", "share": 0.25, "data": "MiD" },
    { "group": "65-74", "purpose": "work", "share": 0.04 + 0.04, "data": "MiD" },
    { "group": "65-74", "purpose": "education", "share": 0.0, "data": "MiD" },
    { "group": "65-74", "purpose": "shop", "share": 0.28, "data": "MiD" },
    { "group": "65-74", "purpose": "leisure", "share": 0.35, "data": "MiD" },
    { "group": "75+", "purpose": "work", "share": 0.01 + 0.0, "data": "MiD" },
    { "group": "75+", "purpose": "education", "share": 0.0, "data": "MiD" },
    { "group": "75+", "purpose": "shop", "share": 0.32, "data": "MiD" },
    { "group": "75+", "purpose": "leisure", "share": 0.33, "data": "MiD" },
])])

In [98]:
px.bar(
    df, x = "group", y = "share", facet_col = "data", barmode = "stack",
    title = "Purpose share by group", color = "purpose"
)

### Total daily trips and distance by mode and purpose (Table 5)

In [99]:
df = trip_zones(df_trips[["trip_index", "mode"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "mode"
]).size().reset_index(name = "trips").assign(data = "synthetic")
df["trips"] /= sampling_factor

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "trips": 2 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "trips": 1.4 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "trips": 3.1 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "trips": 1.1 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "trips": 1.7 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "trips": 1.2 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "trips": 0.9 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "trips": 1.1 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "trips": 0.5 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "trips": 1.2 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "trips": 0.8 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "trips": 0.6 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "trips": 2.0 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "trips": 0.7 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "trips": 0.5 * 1e6, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "trips", pattern_shape = "data", barmode = "group",
    title = "Daily trips per mode", color = "mode"
)

In [100]:
df_sum = df.groupby(["zone_id", "data"])["trips"].sum().reset_index().sort_values(by = "data", ascending = False)
px.bar(
    df_sum, x = "zone_id", y = "trips", pattern_shape = "data", barmode = "group",
    title = "Daily trips in total"
)

In [101]:
df = trip_zones(df_trips[["trip_index", "purpose"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "purpose"
]).size().reset_index(name = "trips").assign(data = "synthetic")
df["trips"] /= sampling_factor

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "purpose": "work", "trips": (1.6 + 0.9) * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "education", "trips": 0.7 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "shop", "trips": 1.5 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "leisure", "trips": 2.7 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "work", "trips": (0.9 + 0.4) * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "education", "trips": 0.3 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "shop", "trips": 0.8 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "leisure", "trips": 1.4 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "work", "trips": (0.7 + 0.5) * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "education", "trips": 0.4 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "shop", "trips": 0.7 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "leisure", "trips": 1.3 * 1e6, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "trips", pattern_shape = "data", barmode = "group",
    title = "Daily trips per purpose", color = "purpose"
)

In [102]:
df = trip_zones(df_trips[["trip_index", "mode", "distance_km"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "mode"
])["distance_km"].sum().reset_index(name = "distance_km").assign(data = "synthetic")
df["distance_km"] /= sampling_factor

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "mode": "walk", "distance_km": 3.1 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "bike", "distance_km": 5.4 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car", "distance_km": 56.3 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "car_passenger", "distance_km": 24.4 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "mode": "pt", "distance_km": 34.4 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "walk", "distance_km": 1.7 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "bike", "distance_km": 3.1 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "car", "distance_km": 21.4 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "car_passenger", "distance_km": 12.0 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "mode": "pt", "distance_km": 21.9 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "walk", "distance_km": 1.4 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "bike", "distance_km": 2.3 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "car", "distance_km": 34.9 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "car_passenger", "distance_km": 12.4 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "mode": "pt", "distance_km": 12.5 * 1e6, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Daily distance per mode", color = "mode"
)

In [103]:
df = trip_zones(df_trips[["trip_index", "purpose", "distance_km"]], ["mvv", "munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "purpose"
])["distance_km"].sum().reset_index(name = "distance_km").assign(data = "synthetic")
df["distance_km"] /= sampling_factor

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "mvv", "purpose": "work", "distance_km": (25.6 + 20.3) * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "education", "distance_km": 4.5 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "shop", "distance_km": 6.5 * 1e6, "data": "MiD" },
    { "zone_id": "mvv", "purpose": "leisure", "distance_km": 48.3 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "work", "distance_km": (11.1 + 10.2) * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "education", "distance_km": 1.9 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "shop", "distance_km": 2.8 * 1e6, "data": "MiD" },
    { "zone_id": "munich", "purpose": "leisure", "distance_km": 48.3 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "work", "distance_km": (14.5 + 10.1) * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "education", "distance_km": 2.6 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "shop", "distance_km": 3.6 * 1e6, "data": "MiD" },
    { "zone_id": "umland", "purpose": "leisure", "distance_km": 22.9 * 1e6, "data": "MiD" },
])])

px.bar(
    df, x = "zone_id", y = "distance_km", pattern_shape = "data", barmode = "group",
    title = "Daily distance per purpose", color = "purpose"
)

### Departure times (Figures 31, 32)

In [104]:
def departure_slots(df):
    df_update = df.copy()

    df_update["departure_slot"] = "22-5"
    df_update.loc[df_update["departure_time"].between(5.0 * 3600.0, 8.0 * 3600.0, inclusive = "left"), "departure_slot"] = "5-8"
    df_update.loc[df_update["departure_time"].between(8.0 * 3600.0, 10.0 * 3600.0, inclusive = "left"), "departure_slot"] = "8-10"
    df_update.loc[df_update["departure_time"].between(10.0 * 3600.0, 13.0 * 3600.0, inclusive = "left"), "departure_slot"] = "10-13"
    df_update.loc[df_update["departure_time"].between(13.0 * 3600.0, 16.0 * 3600.0, inclusive = "left"), "departure_slot"] = "13-16"
    df_update.loc[df_update["departure_time"].between(16.0 * 3600.0, 19.0 * 3600.0, inclusive = "left"), "departure_slot"] = "16-19"
    df_update.loc[df_update["departure_time"].between(19.0 * 3600.0, 22.0 * 3600.0, inclusive = "left"), "departure_slot"] = "19-22"
    
    return df_update

In [105]:
df = trip_zones(departure_slots(df_trips[["trip_index", "departure_time"]]), ["munich", "umland"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "departure_slot"
]).size().reset_index(name = "count").assign(data = "synthetic")

df["sorter"] = df["departure_slot"].apply(lambda x: int(x.split("-")[0]))
df = df.sort_values(by = "sorter")

df_total = df.groupby("zone_id")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total)
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "zone_id": "munich", "departure_slot": "5-8", "share": 0.11, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "8-10", "share": 0.13, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "10-13", "share": 0.16, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "13-16", "share": 0.20, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "16-19", "share": 0.25, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "19-22", "share": 0.10, "data": "MiD" },
    { "zone_id": "munich", "departure_slot": "22-5", "share": 0.03, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "5-8", "share": 0.14, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "8-10", "share": 0.12, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "10-13", "share": 0.18, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "13-16", "share": 0.23, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "16-19", "share": 0.23, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "19-22", "share": 0.08, "data": "MiD" },
    { "zone_id": "umland", "departure_slot": "22-5", "share": 0.02, "data": "MiD" },
])])

px.bar(
    df, x = "departure_slot", y = "share", pattern_shape = "data", barmode = "group",
    title = "Departure times", facet_col = "zone_id"
).update_xaxes(type = "category")

# MVG Comparison

### Mode share 

In [106]:
df = trip_zones(df_trips[["trip_index", "mode"]], ["mvg_influence"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "mode"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("zone_id")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total)
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "mode": "walk", "share": 0.19, "data": "MVG" },
    { "mode": "car", "share": 0.37, "data": "MVG" },
    { "mode": "car_passenger", "share": 0.13, "data": "MVG" },
    { "mode": "park&ride", "share": 0.01, "data": "MVG" },
    { "mode": "bike", "share": 0.16, "data": "MVG" },
    { "mode": "pt", "share": 0.15, "data": "MVG" },
])])

px.bar(
    df, x = "mode", y = "share", pattern_shape = "data", barmode = "group",
    title = "Mode share by trips"
)

### Mode share by distance class

In [107]:
shares = np.array([
    [90,	10,	0,	0,	0,	0,	0,	0,	0,	0,	0,],
    [23,	22,	10,	11,	12,	8,	5,	3,	4,	2,	0,],
    [31,	26,	9,	10,	10,	5,	3,	2,	2,	2,	0,],
    [2,	3,	8,	7,	10,	12,	11,	9,	17,	21,	0,],
    [64,	21,	6,	4,	2,	1,	1,	0,	0,	0,	0,],
    [13,	25,	17,	11,	13,	8,	5,	3,	3,	3,	0,],
]) * 1e-2

In [108]:
df = df_trips[["trip_index", "mode", "distance_km"]].copy()
bounds = [2.5, 5.0, 7.5, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 60.0, np.inf]
distance_classes = np.array([0] + bounds[:-1])
df["distance_class"] = distance_classes[np.digitize(df["distance_km"], bounds)]

df = trip_zones(df, ["mvg_influence"]).drop(columns = ["trip_index"]).groupby([
    "zone_id", "mode", "distance_class"
]).size().reset_index(name = "count").assign(data = "synthetic")

df_total = df.groupby("mode")["count"].sum().reset_index(name = "total")
df = pd.merge(df, df_total)
df["share"] = df["count"] / df["total"]

df = pd.concat([df, pd.DataFrame.from_records([
    { "mode": mode, "distance_class": category, "share": shares[m, c], "data": "MVG" }
    for m, mode in enumerate(["walk", "car", "car_passenger", "park&ride", "bike", "pt"])
    for c, category in enumerate(distance_classes)
])])

df = df.sort_values(by = ["data", "mode", "distance_class"])

px.line(
    df, x = "distance_class", y = "share", color = "mode", line_dash = "data",
    title = "Mode share by trips"
)